In [1]:
import pickle

with open('data/SEGTHOR/spacing.pkl', 'rb') as f:
    spacing_data = pickle.load(f)

print(spacing_data)

{'Patient_08': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_06': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_11': (np.float32(1.269531), np.float32(1.269531), np.float32(2.5)), 'Patient_14': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_15': (np.float32(0.896484), np.float32(0.896484), np.float32(2.5)), 'Patient_38': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_23': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_40': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_24': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_18': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_09': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5)), 'Patient_36': (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0)), 'Patient_04': (np.float32(0.976562), np.float32(0.976562), np.float32(2.5

In [26]:
import nibabel as nib
import numpy as np

import nibabel as nib

def get_nii_contents(filename):
    # Load the NIfTI file
    nii_file = nib.load(filename)

    # Get the image data as a numpy array
    return nii_file.get_fdata()

gt = np.array(get_nii_contents("C:/Users/maart/Downloads/GT.nii"))
gt2 = np.array(get_nii_contents("C:/Users/maart/Downloads/GT2.nii"))


In [27]:
gt2

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [23]:
import numpy as np

def generate_all_corresponding_points(image1, image2):
    """
    Generates a list of corresponding points from two 3D images (GT1 and GT2)
    by finding all the points where both images have non-zero voxel values.
    
    Parameters:
    - image1: NumPy array representing GT1 (3D image).
    - image2: NumPy array representing GT2 (3D image).
    
    Returns:
    - points_image1: List of sampled points from image1.
    - points_image2: List of corresponding points from image2.
    """
    # Ensure the images have the same shape
    assert image1.shape == image2.shape, "Images must have the same shape"
    
    # Get the dimensions of the images
    dims = image1.shape
    
    # Initialize lists to store corresponding points
    points_image1 = []
    points_image2 = []
    
    # Loop through the entire 3D volume to collect corresponding non-zero points
    for x in range(dims[0]):
        for y in range(dims[1]):
            for z in range(dims[2]):
                voxel_value_img1 = image1[x, y, z]
                voxel_value_img2 = image2[x, y, z]
                
                # Only keep points where the voxel value is non-zero in both images
                if voxel_value_img1 != 0 and voxel_value_img2 != 0:
                    points_image1.append([x, y, z])
                    points_image2.append([x, y, z])  # Assuming the same corresponding coordinates
    
    return np.array(points_image1), np.array(points_image2)

# Example usage (assuming you have already loaded gt1 and gt2 as numpy arrays):
points_GT1, points_GT2 = generate_all_corresponding_points(gt, gt2)

print(f"Number of corresponding points found: {len(points_GT1)}")


Number of corresponding points found: 218221


In [24]:
import numpy as np

# Convert to homogeneous coordinates by adding a column of ones
ones = np.ones((points_GT1.shape[0], 1))

# Convert points to homogeneous coordinates
points_GT1_homogeneous = np.hstack([points_GT1, ones])
points_GT2_homogeneous = np.hstack([points_GT2, ones])

# Compute the affine transformation matrix
P1_pseudo_inverse = np.linalg.pinv(points_GT1_homogeneous)
affine_matrix = np.dot(points_GT2_homogeneous.T, P1_pseudo_inverse.T)

print("Affine transformation matrix:")
print(affine_matrix)


Affine transformation matrix:
[[ 1.00000000e+00 -2.27075303e-15 -5.42968448e-16 -7.82485188e-13]
 [ 1.13208054e-14  1.00000000e+00 -1.43635104e-15 -2.06945572e-12]
 [ 4.97518693e-15 -6.18862600e-16  1.00000000e+00 -4.39648318e-14]
 [ 2.11419424e-17 -2.16637147e-17  2.58378930e-17  1.00000000e+00]]


In [38]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

def apply_affine_transform(image_data, affine_matrix):
    """
    Apply an affine transformation to a 3D image.

    Parameters:
    - image_data: NumPy array representing the 3D image to be transformed.
    - affine_matrix: The affine transformation matrix (4x4).
    
    Returns:
    - transformed_image: The transformed image as a NumPy array.
    """
    # Extract the 3x3 matrix for rotation/scaling
    rotation_scaling_matrix = affine_matrix[:3, :3]
    
    # Extract the translation vector
    translation_vector = affine_matrix[:3, 3]
    
    # Apply the affine transformation to the image data
    transformed_image = affine_transform(image_data, rotation_scaling_matrix, offset=translation_vector)

    return transformed_image


# Example usage
# image_data should be a 3D numpy array, affine_matrix a 4x4 matrix
transformed_gt1 = apply_affine_transform(gt, affine_matrix)

# Print the shape of the transformed image
print("Transformed image shape:", transformed_gt1.shape)


Transformed image shape: (512, 512, 213)


In [39]:
transformed_gt1 == gt2

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True, False, ..., False, False,  True],
        [ True, False, False, ..., False, False,  True],
        ...,
        [ True, False, False, ..., False,  True,  True],
        [ True, False, False, ..., False, False,  True],
        [ True, False, False, ..., False, False,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True, False,  True, ..., False, False,  True],
        [ True, False,  True, ..., False, False,  True],
        ...,
        [ True, False, False, ..., False, False,  True],
        [ True, False, False, ..., False, Fal

In [37]:
gt2

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [40]:
pip install SimpleITK


   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB 653.6 kB/s eta 0:00:29
    --------------------------------------- 0.4/18.7 MB 4.4 MB/s eta 0:00:05
   -- ------------------------------------- 1.2/18.7 MB 8.7 MB/s eta 0:00:03
   -- ------------------------------------- 1.2/18.7 MB 8.7 MB/s eta 0:00:03
   ---- ----------------------------------- 2.0/18.7 MB 9.3 MB/s eta 0:00:02
   ------ --------------------------------- 2.9/18.7 MB 10.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.7/18.7 MB 12.0 MB/s eta 0:00:02
   --------- ------------------------------ 4.6/18.7 MB 12.8 MB/s eta 0:00:02
   ----------- ---------------------------- 5.3/18.7 MB 12.9 MB/s eta 0:00:02
   ------------- -------------------------- 6.2/18.7 MB 13.7 MB/s eta 0:00:01
   --------------- ------------------------ 7.0/18.7 MB 14.1 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import SimpleITK as sitk
import nibabel as nib
import numpy as np


def get_nii_contents(filename):
    """
    Load a NIfTI file and return the image data as a SimpleITK image and NumPy array.
    
    Parameters:
    - filename: Path to the NIfTI file.
    
    Returns:
    - sitk_image: SimpleITK image object for registration.
    - np_image: NumPy array of image data.
    """
    # Load the NIfTI file with nibabel
    nii_file = nib.load(filename)
    
    # Get the image data as a NumPy array
    np_image = np.array(nii_file.get_fdata())
    
    # Convert the NumPy array to a SimpleITK image
    sitk_image = sitk.GetImageFromArray(np_image)
    
    # Set the spacing and origin from NIfTI (important for medical images)
    # sitk_image.SetSpacing(nii_file.header.get_zooms())
    
    return sitk_image, np_image


def register_affine(fixed_image, moving_image):
    """
    Registers the moving image to the fixed image using an affine transformation.
    
    Parameters:
    - fixed_image: SimpleITK image (reference/fixed image).
    - moving_image: SimpleITK image (moving image to be transformed).
    
    Returns:
    - affine_transform: The resulting affine transformation.
    - transformed_image: The transformed moving image (in NumPy format).
    """
    # Set up the registration method
    registration_method = sitk.ImageRegistrationMethod()
    
    # Use mutual information metric for evaluation
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    
    # Set the initial transform with a centered initializer for a better start
    initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                          moving_image, 
                                                          sitk.AffineTransform(fixed_image.GetDimension()), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    
    # Set the optimizer
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=500)
    
    # Set interpolator
    registration_method.SetInterpolator(sitk.sitkLinear)
    
    # Optionally add verbosity to see if it converges
    registration_method.SetOptimizerScalesFromJacobian()
    registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=1.0,
                                                                 minStep=1e-6,
                                                                 numberOfIterations=500,
                                                                 gradientMagnitudeTolerance=1e-6,
                                                                 relaxationFactor=0.5,
                                                                 maximumStepSizeInPhysicalUnits=1.0)
    
    # Execute the registration
    affine_transform = registration_method.Execute(fixed_image, moving_image)
    
    # Apply the transform to the moving image
    transformed_image_sitk = sitk.Resample(moving_image, fixed_image, affine_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
    
    # Convert the transformed SimpleITK image back to a NumPy array
    transformed_image_np = sitk.GetArrayFromImage(transformed_image_sitk)
    
    return affine_transform, transformed_image_np


# Load the images
gt_sitk, gt_np = get_nii_contents("C:/Users/maart/Downloads/GT.nii")
gt2_sitk, gt2_np = get_nii_contents("C:/Users/maart/Downloads/GT2.nii")

# Perform the affine registration
affine_transform, transformed_gt = register_affine(gt_sitk, gt2_sitk)

# Print the affine transformation matrix
print("Affine transformation parameters:")
print(affine_transform)

# Save the transformed image to a NIfTI file (using affine from gt2 for consistency)
transformed_gt_img = nib.Nifti1Image(transformed_gt, nib.load("C:/Users/maart/Downloads/GT2.nii").affine)
nib.save(transformed_gt_img, "C:/Users/maart/Downloads/transformed_GT.nii")

# Print the shape of the transformed image
print("Transformed image shape:", transformed_gt.shape)


Affine transformation parameters:
itk::simple::CompositeTransform
 CompositeTransform (00000120E9A21E40)
   RTTI typeinfo:   class itk::CompositeTransform<double,3>
   Reference Count: 1
   Modified Time: 6541
   Debug: Off
   Object Name: 
   Observers: 
     none
   TransformQueue: 
   >>>>>>>>>
   AffineTransform (00000120FE8F6C60)
     RTTI typeinfo:   class itk::AffineTransform<double,3>
     Reference Count: 1
     Modified Time: 6247
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       1 0 0 
       0 1 0 
       0 0 1 
     Offset: [0, 0, 0]
     Center: [106, 255.5, 255.5]
     Translation: [0, 0, 0]
     Inverse: 
       1 0 0 
       0 1 0 
       0 0 1 
     Singular: 0
   TransformsToOptimizeFlags: 
           1 
   TransformsToOptimizeQueue: 
   PreviousTransformsToOptimizeUpdateTime: 0

Transformed image shape: (512, 512, 213)


In [5]:
import SimpleITK as sitk
import numpy as np
import nibabel as nib
import os

def read_nifti_file(file_path):
    """Read a Nifti file using nibabel and convert to SimpleITK format."""
    img = nib.load(file_path)
    img_data = img.get_fdata()

    # Convert to SimpleITK image
    sitk_img = sitk.GetImageFromArray(img_data)

    # Convert zooms (voxel spacing) to a list and set spacing
    spacing = list(img.header.get_zooms())
    sitk_img.SetSpacing(spacing)

    # Set the origin from the affine transformation matrix
    sitk_img.SetOrigin(img.affine[:3, 3])

    # Set the direction using the rotation part of the affine matrix
    sitk_img.SetDirection(np.reshape(img.affine[:3, :3], 9))

    return sitk_img


def affine_registration(fixed_image, moving_image):
    """Perform affine registration using elastix via SimpleITK."""
    parameter_map = sitk.GetDefaultParameterMap("affine")  # Create an affine parameter map
    parameter_map['FinalBSplineInterpolationOrder'] = ['1']  # Set BSpline interpolation order

    # Elastix setup
    elastix_image_filter = sitk.ElastixImageFilter()
    elastix_image_filter.SetFixedImage(fixed_image)
    elastix_image_filter.SetMovingImage(moving_image)
    elastix_image_filter.SetParameterMap(parameter_map)

    # Run elastix registration
    elastix_image_filter.Execute()

    # Get the resulting transformation parameter map
    transform_parameter_map = elastix_image_filter.GetTransformParameterMap()[0]
    return transform_parameter_map

def extract_affine_matrix(transform_parameter_map):
    """Extract the affine matrix from the transform parameters."""
    transform_params = np.array([float(x) for x in transform_parameter_map['TransformParameters']])
    affine_matrix = np.zeros((4, 4))

    # The first 9 values form the 3x3 rotation matrix (row-major order)
    affine_matrix[:3, :3] = transform_params[:9].reshape(3, 3)
    
    # The last 3 values are the translation
    affine_matrix[:3, 3] = transform_params[9:12]
    
    # Bottom row of the affine matrix
    affine_matrix[3, :] = [0, 0, 0, 1]
    
    return affine_matrix

def main():
    # Define file paths for gt and gt2
    fixed_image_path = 'C:/Users/maart/Downloads/GT.nii'  # Reference image
    moving_image_path = 'C:/Users/maart/Downloads/GT2.nii'  # Image to be transformed

    # Read Nifti files
    fixed_image = read_nifti_file(fixed_image_path)
    moving_image = read_nifti_file(moving_image_path)

    # Perform affine registration
    transform_params = affine_registration(fixed_image, moving_image)

    # Extract and display the affine matrix
    affine_matrix = extract_affine_matrix(transform_params)
    print("Affine Transformation Matrix:")
    print(affine_matrix)

    # Optionally, save the affine matrix to a file
    np.savetxt('affine_matrix.txt', affine_matrix)


main()


TypeError: in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'

In [2]:
gt_sitk, gt_np = get_nii_contents("C:/Users/maart/Downloads/GT.nii")
gt2_sitk, gt2_np = get_nii_contents("C:/Users/maart/Downloads/GT2.nii")

In [10]:
import SimpleITK as sitk
import numpy as np

# Function to read NIfTI images
def read_nii_file(filepath):
    image = sitk.ReadImage(filepath)
    return image

# Function to calculate the affine transformation between two images
def calculate_affine_transform(fixed_image_path, moving_image_path):
    # Read the NIfTI files
    fixed_image = read_nii_file(fixed_image_path)
    moving_image = read_nii_file(moving_image_path)

    # Set up elastix object
    elastix = sitk.ElastixImageFilter()
    elastix.SetFixedImage(fixed_image)
    elastix.SetMovingImage(moving_image)

    # Set affine transformation parameters
    parameter_map = sitk.GetDefaultParameterMap("affine")
    elastix.SetParameterMap(parameter_map)

    # Perform the registration
    elastix.Execute()

    # Get the result image and transform parameters
    result_image = elastix.GetResultImage()
    transform_parameters = elastix.GetTransformParameterMap()

    return transform_parameters

# Function to extract and print the affine transformation matrix
def extract_affine_matrix(transform_parameters):
    # The affine matrix is stored in the 'TransformParameters' key of the parameter map
    affine_parameters = np.array([float(p) for p in transform_parameters[0]['TransformParameters']])

    # The affine matrix for 3D data is typically stored in the first 9 values (3x3 matrix)
    affine_matrix = affine_parameters[:9].reshape(3, 3)

    # Extract translation (last 3 values)
    translation_vector = affine_parameters[9:]

    return affine_matrix, translation_vector

# Paths to the NIfTI files (replace these with your actual file paths)
fixed_image_path = "C:/Users/maart/Downloads/GT.nii"
moving_image_path = "C:/Users/maart/Downloads/GT2.nii"

# Perform the affine transformation calculation
transform_parameters = calculate_affine_transform(fixed_image_path, moving_image_path)

# Extract and display the affine matrix
affine_matrix, translation_vector = extract_affine_matrix(transform_parameters)

print("Affine Matrix:\n", affine_matrix)
print("Translation Vector:\n", translation_vector)


AttributeError: module 'SimpleITK' has no attribute 'ElastixImageFilter'

In [2]:
import itk
import nibabel as nib
import numpy as np

def read_nii_file(filename):
    # Load the NIfTI file
    nii_file = nib.load(filename)
    image = itk.image_from_array(np.array(nii_file.get_fdata()))
    return image

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
default_affine_parameter_map = parameter_object.GetDefaultParameterMap('affine',3)
default_affine_parameter_map['FinalBSplineInterpolationOrder'] = ['0']
parameter_object.AddParameterMap(default_affine_parameter_map)

fixed_image_path = "C:/Users/maart/Downloads/GT.nii"
moving_image_path = "C:/Users/maart/Downloads/GT2.nii"
fixed_image = read_nii_file(fixed_image_path)
moving_image = read_nii_file(moving_image_path)

: 

In [3]:
# Call registration function
result_image_affine, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object,
    log_to_console=True)

In [3]:
print(f"Fixed image size: {fixed_image.GetLargestPossibleRegion().GetSize()}")
print(f"Moving image size: {moving_image.GetLargestPossibleRegion().GetSize()}")


Fixed image size: itkSize3 ([512, 512, 213])
Moving image size: itkSize3 ([512, 512, 213])


In [4]:
import SimpleITK as sitk

def register_affine():
    """
    Perform affine registration using SimpleITK.
    
    Args:
    - fixed_image_path: Path to the fixed (correct ground truth) image.
    - moving_image_path: Path to the moving (incorrect ground truth) image.
    
    Returns:
    - transform: The resulting affine transformation (SimpleITK Transform object).
    """
    # Read the fixed (correct) and moving (incorrect) images
    # Load incorrect and correct ground truth (GT) images
    fixed_image = sitk.ReadImage("C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT2.nii.gz", sitk.sitkFloat32)   # Correct GT
    moving_image = sitk.ReadImage("C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT.nii.gz", sitk.sitkFloat32)  # Incorrect GT
    # fixed_image = sitk.ReadImage(fixed_image_path, sitk.sitkFloat32)
    # moving_image = sitk.ReadImage(moving_image_path, sitk.sitkFloat32)

    # Initialize the registration method
    registration_method = sitk.ImageRegistrationMethod()

    # Set up the similarity metric
    registration_method.SetMetricAsMeanSquares()  # You can use other metrics like MutualInformation

    # Set up the interpolator
    registration_method.SetInterpolator(sitk.sitkLinear)

    # Initialize the transformation (Affine)
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        moving_image,
        sitk.AffineTransform(fixed_image.GetDimension()),  # Use affine transform
        sitk.CenteredTransformInitializerFilter.GEOMETRY
    )

    # Set the initial transform
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    # Set up the optimizer
    registration_method.SetOptimizerAsRegularStepGradientDescent(
        learningRate=1.0,
        minStep=1e-6,
        numberOfIterations=200,
        gradientMagnitudeTolerance=1e-6
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Execute the registration
    final_transform = registration_method.Execute(fixed_image, moving_image)

    print("Optimizer's stopping condition: {0}".format(registration_method.GetOptimizerStopConditionDescription()))
    print("Final metric value: {0}".format(registration_method.GetMetricValue()))

    return final_transform

def apply_affine_transform_to_class(segmented_image, transform, output_image_path):
    """
    Applies an affine transformation to a specified isolated class in a segmented image.
    
    Args:
    - segmented_image: The already segmented image with the class isolated.
    - transform: The affine transformation to be applied.
    - output_image_path: Path to save the transformed image.
    """
    # Apply the affine transformation to the segmented image
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(segmented_image)
    resampler.SetTransform(transform)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)  # Use nearest neighbor to preserve label values
    transformed_image = resampler.Execute(segmented_image)  # Apply transformation to the entire image
    
    # Save the transformed image as a new image
    sitk.WriteImage(transformed_image, output_image_path)


# Example usage
# fixed_image_path = r"C:\path\to\correct_GT.nii.gz"
# moving_image_path = r"C:\path\to\incorrect_GT.nii.gz"
affine_transform = register_affine()

# Example usage
output_image_path = r"C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/transformed_image.nii.gz"

# Load the already isolated segmented image
segmented_image = sitk.ReadImage("C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT.nii.gz")

# Apply the transform to the already isolated class
apply_affine_transform_to_class(segmented_image, affine_transform, output_image_path)

Optimizer's stopping condition: RegularStepGradientDescentOptimizerv4: Gradient magnitude tolerance met after 0 iterations. Gradient magnitude (0) is less than gradient magnitude tolerance (1e-06).
Final metric value: 0.0


In [8]:
import SimpleITK as sitk

def isolate_heart(gt_image_path, heart_label=2):
    """
    Isolate the heart region from the ground truth segmentation image.
    
    Args:
    - gt_image_path: Path to the ground truth segmentation image.
    - heart_label: The label value that corresponds to the heart (default is 1).
    
    Returns:
    - isolated_heart: The binary image containing only the heart region.
    """
    # Load the ground truth segmentation image
    gt_image = sitk.ReadImage(gt_image_path)
    
    # Use a threshold filter to isolate the heart (label = heart_label)
    threshold_filter = sitk.BinaryThresholdImageFilter()
    threshold_filter.SetLowerThreshold(heart_label)
    threshold_filter.SetUpperThreshold(heart_label)
    threshold_filter.SetInsideValue(1)  # Set the heart region to 1
    threshold_filter.SetOutsideValue(0)  # Set the rest to 0
    
    # Apply the threshold to create a binary mask of the heart
    isolated_heart = threshold_filter.Execute(gt_image)
    
    return isolated_heart

# Example usage to isolate the heart from GT2 (fixed image)
gt2_image_path = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/GT2.nii.gz"
isolated_heart_image_gt2 = isolate_heart(gt2_image_path)

# Save the isolated heart from GT2 as a new image
output_image_path_gt2 = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT2.nii.gz"
sitk.WriteImage(isolated_heart_image_gt2, output_image_path_gt2)

# Example usage to isolate the heart from GT (moving image)
gt_image_path = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/GT.nii.gz"
isolated_heart_image_gt = isolate_heart(gt_image_path)

# Save the isolated heart from GT as a new image
output_image_path_gt = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT.nii.gz"
sitk.WriteImage(isolated_heart_image_gt, output_image_path_gt)


In [9]:
import SimpleITK as sitk

def register_affine():
    """
    Perform affine registration using SimpleITK.
    
    Returns:
    - final_transform: The resulting affine transformation (SimpleITK Transform object).
    """
    # Read the fixed (correct) and moving (incorrect) images
    fixed_image = sitk.ReadImage("C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT2.nii.gz", sitk.sitkFloat32)   # Correct GT
    moving_image = sitk.ReadImage("C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT.nii.gz", sitk.sitkFloat32)  # Incorrect GT

    # Initialize the registration method
    registration_method = sitk.ImageRegistrationMethod()

    # Set up the similarity metric
    registration_method.SetMetricAsMeanSquares()  # Use MeanSquares as metric

    # Set up the interpolator
    registration_method.SetInterpolator(sitk.sitkLinear)  # Linear interpolation for registration

    # Initialize the transformation (Affine)
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        moving_image,
        sitk.AffineTransform(fixed_image.GetDimension()),  # Use affine transform
        sitk.CenteredTransformInitializerFilter.GEOMETRY
    )

    # Set the initial transform
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    # Set up the optimizer
    registration_method.SetOptimizerAsRegularStepGradientDescent(
        learningRate=1.0,
        minStep=1e-6,
        numberOfIterations=200,
        gradientMagnitudeTolerance=1e-6
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Execute the registration
    final_transform = registration_method.Execute(fixed_image, moving_image)

    # Output optimizer and metric information
    print("Optimizer's stopping condition: {0}".format(registration_method.GetOptimizerStopConditionDescription()))
    print("Final metric value: {0}".format(registration_method.GetMetricValue()))

    return final_transform

def apply_affine_transform_to_class(segmented_image, transform, output_image_path):
    """
    Applies an affine transformation to a specified isolated class in a segmented image.
    
    Args:
    - segmented_image: The already segmented image with the class isolated.
    - transform: The affine transformation to be applied.
    - output_image_path: Path to save the transformed image.
    """
    # Apply the affine transformation to the segmented image
    resampler = sitk.ResampleImageFilter()
    
    # Set resampling properties to match the segmented image
    resampler.SetReferenceImage(segmented_image)  # Reference the original segmented image
    resampler.SetTransform(transform)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)  # Nearest neighbor to preserve label values
    resampler.SetOutputSpacing(segmented_image.GetSpacing())
    resampler.SetOutputOrigin(segmented_image.GetOrigin())
    resampler.SetOutputDirection(segmented_image.GetDirection())
    
    # Apply the transformation
    transformed_image = resampler.Execute(segmented_image)

    # Save the transformed image as a new image
    sitk.WriteImage(transformed_image, output_image_path)
    print(f"Transformed image saved at: {output_image_path}")


# Example usage
affine_transform = register_affine()

# Specify output path for the transformed image
output_image_path = r"C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/transformed_image.nii.gz"

# Load the already isolated segmented image
segmented_image = sitk.ReadImage("C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/isolated_heart_GT.nii.gz")

# Apply the transform to the already isolated class
apply_affine_transform_to_class(segmented_image, affine_transform, output_image_path)


Optimizer's stopping condition: RegularStepGradientDescentOptimizerv4: Maximum number of iterations (200) exceeded.
Final metric value: 0.001245705170602879
Transformed image saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/transformed_image.nii.gz


In [10]:
sitk.WriteTransform(affine_transform, 'C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train/Patient_27/affine_transform.tfm')


In [12]:
import os
import SimpleITK as sitk
import numpy as np

# Define the directory where patient folders are located
data_dir = "C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train"

# Define the affine transformation (adjust these values as needed)
affine_transform = sitk.AffineTransform(3)
affine_transform.SetParameters([1.0107339043421648, 0.024322574439718903, 0.02479276479728504,
                                0.11594828634373514, 1.009085531494732, -0.05903886252715501,
                                -0.049156567967914684, -0.016809583505448446, 0.9866310801641027,
                                -64.35096518079041, -43.02182351044668, -29.479998868180246])

# Function to apply affine transformation to the heart (class 2)
def apply_affine_transform_to_heart(input_image_path, output_image_path, transform, heart_label=2):
    # Read the input image (GT.nii.gz)
    input_image = sitk.ReadImage(input_image_path)
    
    # Convert the input image to a numpy array
    input_array = sitk.GetArrayFromImage(input_image)
    
    # Isolate the heart (class 2) by creating a binary mask
    heart_mask = (input_array == heart_label).astype(np.uint8)
    
    # Convert the heart mask to SimpleITK image
    heart_image = sitk.GetImageFromArray(heart_mask)
    heart_image.CopyInformation(input_image)
    
    # Apply the affine transformation only to the heart
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(heart_image)
    resampler.SetTransform(transform)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)  # Use nearest neighbor to preserve label values
    
    transformed_heart_image = resampler.Execute(heart_image)
    
    # Convert the transformed heart back to a numpy array
    transformed_heart_array = sitk.GetArrayFromImage(transformed_heart_image)
    
    # Reintegrate the transformed heart back into the original segmentation
    output_array = input_array.copy()
    output_array[transformed_heart_array == 1] = heart_label
    
    # Convert the output array back to a SimpleITK image
    output_image = sitk.GetImageFromArray(output_array)
    output_image.CopyInformation(input_image)
    
    # Write the transformed image to the output file
    sitk.WriteImage(output_image, output_image_path)
    print(f"Transformed heart saved at: {output_image_path}")

# Loop through all patient directories
for patient_folder in os.listdir(data_dir):
    patient_path = os.path.join(data_dir, patient_folder)
    
    # Check if the directory contains the GT.nii.gz file
    gt_path = os.path.join(patient_path, "GT.nii.gz")
    
    if os.path.isfile(gt_path):
        # Define the output path for the transformed image
        output_path = os.path.join(patient_path, "GT_transformed.nii.gz")
        
        # Apply the transformation to the heart and save the result
        apply_affine_transform_to_heart(gt_path, output_path, affine_transform)
    else:
        print(f"GT.nii.gz not found for {patient_folder}")


Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_01\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_02\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_03\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_04\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_05\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_06\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_07\GT_transformed.nii.gz
Transformed heart saved at: C:/Users/maart/Documents/ai4mi_project/data/segthor_train/train\Patient_08\GT_transformed.nii.gz
